In [1]:
from kgevaluator import KGEvaluator
from ontology.loader import PEKGOntology
import networkx as nx

In [6]:
kg_path = "C:/PE/outputs/DECK/knowledge_graph_DECK_gpt-4.1_iterative.json"
ontology_path = "C:/PE/REPOS/llm_kg_extraction/llm_kg_extraction/ontology/pekg_ontology.yaml"

kge = KGEvaluator(kg_path, ontology_path)

In [6]:
metrics = kge.basic_metrics()

In [7]:
quality = kge.evaluate_graph_quality()

In [8]:
largest_cc = max(nx.weakly_connected_components(kge.graph), key=len)
subgraph = kge.graph.subgraph(largest_cc)

In [9]:
nx.diameter(subgraph.to_undirected())

4

In [10]:
distribution_relations = kge.relation_distribution()
distribution_entities = kge.entity_coverage()

In [11]:
distribution_relations

({'hasCustomer': 0.6296296296296297,
  'contractsWithGov': 0.25925925925925924,
  'advisorFor': 0.037037037037037035,
  'hasExecutive': 0.037037037037037035,
  'hasMembership': 0.037037037037037035},
 {'hasCustomer': 17,
  'contractsWithGov': 7,
  'advisorFor': 1,
  'hasExecutive': 1,
  'hasMembership': 1})

In [12]:
distribution_entities

({'Company': True,
  'LegalEntity': False,
  'Person': True,
  'Department': False,
  'Position': False,
  'Committee': True,
  'Product': False,
  'UseCase': False,
  'Investor': False,
  'Advisor': False,
  'GovernmentBody': True,
  'FinancialInstrument': False,
  'OwnershipStake': False,
  'Contract': False,
  'IntellectualProperty': False,
  'Litigation': False,
  'Risk': False,
  'RegulatoryRequirement': False,
  'PolicyDocument': False,
  'CorporateEvent': False,
  'FundingRound': False,
  'AcquisitionEvent': False,
  'MergerEvent': False,
  'IPOEvent': False,
  'ExitEvent': False,
  'LeadershipChangeEvent': False,
  'FinancialMetric': False,
  'HeadcountMetric': False,
  'MarketMetric': False,
  'TaxMetric': False,
  'KPI': False,
  'NewsItem': False,
  'Location': False,
  'overall_entity_coverage': 0.12121212121212122},
 defaultdict(int,
             {'Company': 20,
              'GovernmentBody': 6,
              'Person': 1,
              'Committee': 1,
              'Legal

In [13]:
quality != kge.evaluate_graph_quality()
quality

{'isolated_nodes_ratio': 0.0,
 'attribute_completeness': 1.0,
 'financial_metrics_availability': True,
 'avg_company_relations': 2.25,
 'graph_diameter': 4}

In [14]:
ontology_compliance = kge.evaluate_ontology_compliance()
ontology_compliance

{'entity_type_validity': 1.0,
 'relation_type_validity': 0.9259259259259259,
 'domain_range_validity': 0.8888888888888888,
 'attribute_validity': 0.0,
 'overall_compliance': 0.7037037037037037,
 'validation_details': {'invalid_entities': [],
  'invalid_relations': [{'source': 'e26',
    'target': 'e1',
    'invalid_type': 'advisorFor'},
   {'source': 'e27', 'target': 'e28', 'invalid_type': 'hasMembership'}],
  'domain_range_violations': [{'relation_type': 'contractsWithGov',
    'source': {'id': 'e1', 'type': 'Company'},
    'target': {'id': 'e25', 'type': 'Company'},
    'expected_domain': ['Company'],
    'expected_range': ['GovernmentBody']}]}}

In [3]:
kge.evaluate()

{'basic_metrics': {'num_nodes': 28,
  'num_edges': 27,
  'density': 0.03571428571428571,
  'avg_degree': 1.9285714285714286,
  'connected_components': 1,
  'node_type_distribution': {'Company': 20,
   'GovernmentBody': 6,
   'Person': 1,
   'Committee': 1}},
 'ontology_compliance': {'entity_type_validity': 1.0,
  'relation_type_validity': 0.9259259259259259,
  'domain_range_validity': 0.8888888888888888,
  'attribute_validity': 0.0,
  'overall_compliance': 0.7037037037037037,
  'validation_details': {'invalid_entities': [],
   'invalid_relations': [{'source': 'e26',
     'target': 'e1',
     'invalid_type': 'advisorFor'},
    {'source': 'e27', 'target': 'e28', 'invalid_type': 'hasMembership'}],
   'domain_range_violations': [{'relation_type': 'contractsWithGov',
     'source': {'id': 'e1', 'type': 'Company'},
     'target': {'id': 'e25', 'type': 'Company'},
     'expected_domain': ['Company'],
     'expected_range': ['GovernmentBody']}]}},
 'graph_quality': {'isolated_nodes_ratio': 0.0

In [7]:
kge.evaluate()

{'basic_metrics': {'num_nodes': 47,
  'num_edges': 47,
  'density': 0.021739130434782608,
  'avg_degree': 2.0,
  'connected_components': 1,
  'node_type_distribution': {'Company': 22,
   'Product': 1,
   'UseCase': 3,
   'GovernmentBody': 7,
   'FinancialMetric': 2,
   'HeadcountMetric': 1,
   'Person': 1,
   'LegalEntity': 1,
   'Position': 1,
   'Committee': 1,
   'Location': 4,
   'Contract': 1,
   'CorporateEvent': 1,
   'NewsItem': 1}},
 'ontology_compliance': {'entity_type_validity': 1.0,
  'relation_type_validity': 0.9148936170212766,
  'domain_range_validity': 0.7659574468085106,
  'attribute_validity': 1.0,
  'overall_compliance': 0.9202127659574468,
  'validation_details': {'invalid_entities': [],
   'invalid_relations': [{'source': 'e1',
     'target': 'e2',
     'invalid_type': 'hasProduct'},
    {'source': 'e1', 'target': 'e30', 'invalid_type': 'competesWith'},
    {'source': 'e1', 'target': 'e31', 'invalid_type': 'competesWith'},
    {'source': 'e45', 'target': 'e35', 'in